In [1]:
import numpy as np
import pandas as pd
from scipy.special import comb

from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from sklearn.model_selection import (
    KFold,
    RepeatedKFold,
    StratifiedKFold,
    LeaveOneOut,
    LeavePOut,
    cross_validate,
    train_test_split
)

In [2]:
breast_cancer_X, breast_cancer_y = load_breast_cancer(return_X_y=True)
X = pd.DataFrame(breast_cancer_X)
y = pd.Series(breast_cancer_y).map({0: 1, 1: 0})# Map nesse caso serve para inverter os valores
X.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


## K-Fold Cross-Validation

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [10]:
from sklearn.model_selection import GridSearchCV

#Logistic Regression   
logit = LogisticRegression(solver='liblinear', random_state=42, C=1, max_iter=100000, penalty='l2')

#Hyperparameters space
param_grid = dict(
    penalty=['l1', 'l2'],
    C=[0.1, 1, 10, 100]
)

#K-Fold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

#Grid Search
clf = GridSearchCV(
    logit,
    param_grid,
    cv=kf, 
    scoring='accuracy',
    refit=True
)

search = clf.fit(X_train, y_train)

#Results
search.best_params_

{'C': 100, 'penalty': 'l1'}

In [13]:
results = pd.DataFrame(search.cv_results_)[['params', 'mean_test_score', 'std_test_score']]
print(results.shape)
results

(8, 3)


,params,mean_test_score,std_test_score
0,"{'C': 0.1, 'penalty': 'l1'}",0.923077,0.019658
1,"{'C': 0.1, 'penalty': 'l2'}",0.931868,0.010767
2,"{'C': 1, 'penalty': 'l1'}",0.953846,0.012815
3,"{'C': 1, 'penalty': 'l2'}",0.956044,0.012038
4,"{'C': 10, 'penalty': 'l1'}",0.958242,0.016150
5,"{'C': 10, 'penalty': 'l2'}",0.951648,0.014906
6,"{'C': 100, 'penalty': 'l1'}",0.964835,0.010767
7,"{'C': 100, 'penalty': 'l2'}",0.953846,0.014579


In [12]:
#Vizualização dos resultados
results.sort.values(by='mean_test_score', ascending=False, inplace = True) 
results.reset_index(drop=True, inplace=True)
results

AttributeError: 'DataFrame' object has no attribute 'sort'

In [14]:
#Pegando as predições
train_preds = search.predict(X_train)
test_preds = search.predict(X_test)

#Acurácia
print('Train Accuracy:', accuracy_score(y_train, train_preds))
print('Test Accuracy:', accuracy_score(y_test, test_preds))

Train Accuracy: 0.989010989010989
Test Accuracy: 0.9824561403508771
